In [1]:
!pip install pandas-datareader

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

tf.__version__

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


'2.3.0'

In [3]:
class DQN_trader():
  
  def __init__(self, state_size, action_num=3, model_name="DQN_trader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_num = action_num
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_dnn()
    
  def model_dnn(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=16, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_num, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=1e-3))
    
    return model
  
  def trade(self, state):
    # random or use model predict
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_num)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    # get pervious action
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [4]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [5]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close


def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [6]:
stock_name = "TSM"
data = dataset_loader(stock_name)
data_reader.DataReader(stock_name, data_source="yahoo").head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-09-14,20.170000,19.889999,19.990000,20.040001,7790800.0,16.942183
2015-09-15,20.559999,20.040001,20.160000,20.389999,11469800.0,17.238077
2015-09-16,20.750000,20.459999,20.709999,20.670000,11471100.0,17.474792
2015-09-17,21.020000,20.570000,20.580000,20.660000,11394200.0,17.466341
2015-09-18,20.760000,20.350000,20.530001,20.610001,14057800.0,17.424070


In [7]:
window_size = 10
episodes = 100

batch_size = 32
data_samples = len(data) - 1

In [8]:
trader = DQN_trader(window_size)

In [9]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 387       
Total params: 11,539
Trainable params: 11,539
Non-trainable params: 0
_________________________________________________________________


In [10]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("DQN Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("DQN Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/100
DQN Trader bought:  $ 20.040001
DQN Trader bought:  $ 20.660000
DQN Trader sold:  $ 20.530001  Profit: $ 0.490000
DQN Trader sold:  $ 20.010000  Profit: - $ 0.650000
DQN Trader bought:  $ 19.990000
DQN Trader bought:  $ 20.049999
DQN Trader bought:  $ 19.629999
DQN Trader bought:  $ 19.889999
DQN Trader sold:  $ 20.750000  Profit: $ 0.760000
DQN Trader bought:  $ 20.860001
DQN Trader bought:  $ 21.400000
DQN Trader bought:  $ 21.230000
DQN Trader sold:  $ 22.250000  Profit: $ 2.200001
DQN Trader sold:  $ 21.950001  Profit: $ 2.320002
DQN Trader sold:  $ 22.090000  Profit: $ 2.200001
DQN Trader sold:  $ 21.950001  Profit: $ 1.090000
DQN Trader bought:  $ 22.160000
DQN Trader sold:  $ 22.219999  Profit: $ 0.820000
DQN Trader bought:  $ 22.510000
DQN Trader sold:  $ 22.469999  Profit: $ 1.240000
DQN Trader bought:  $ 22.080000


  3%|▎         | 33/1258 [00:04<03:01,  6.74it/s]

DQN Trader sold:  $ 21.820000  Profit: - $ 0.340000


  3%|▎         | 34/1258 [00:07<18:52,  1.08it/s]

...

DQN Trader bought:  $ 21.230000


  1%|▏         | 17/1258 [00:48<56:40,  2.74s/it]

DQN Trader sold:  $ 21.930000  Profit: $ 0.700001


  3%|▎         | 39/1258 [01:47<55:34,  2.74s/it]

KeyboardInterrupt: ignored